# Object Tracking

Using OpenCV functions I have explored some object tracking techniques and created window-applications for the same

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [2]:
# Good features to track Parameters: Setup as a dict
corner_track_params = dict(maxCorners = 10, qualityLevel = 0.4, minDistance = 7, blockSize = 7)

In [3]:
# Lukas - Kanade params:
lk_params = dict(winSize = (200, 200), maxLevel = 2, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

## Optical Flow

The optical flow methods in OpenCV will first take in a given set of points and a frame. Then it will attempt to find those points in the NEXT frame.

In [5]:
# Capture the video
cap = cv2.VideoCapture(0)

# Grab the very first frame of the stream
ret, prev_frame = cap.read()

# Convert this to grayscale
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# Grabbing the corners of the first frame using good features to track
prevPts = cv2.goodFeaturesToTrack(prev_gray, mask = None, **corner_track_params)

# Create a matching mask of the previous frame for drawing on later
mask = np.zeros_like(prev_frame)


while True:
    # Grab current frame
    ret,frame = cap.read()
    
    # and convert to grayscale
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Calculate the OPTICAL FLOW on the Gray Scale Frame
    nextPts, status, err = cv2.calcOpticalFlowPyrLK(prevImg = prev_gray, nextImg = frame_gray, 
                                                    prevPts = prevPts, nextPts = None, **lk_params)
    
    # Using the returned status array (the status output)
    # status output status vector (of unsigned chars); each element of the vector is set to 1 if
    # the flow for the corresponding features has been found, otherwise, it is set to 0.
    good_new = nextPts[status == 1]
    good_prev = prevPts[status == 1]
    
    # Use ravel to get points to draw lines and circles
    for i, (new, prev) in enumerate(zip(good_new, good_prev)):
        
        x_new, y_new = new.ravel()
        x_prev, y_prev = prev.ravel()
        
        # Lines will be drawn using the mask created from the first frame
        mask = cv2.line(mask, (x_new, y_new), (x_prev, y_prev), (0, 255, 0), 3)
        
        # Draw red circles at corner points
        frame = cv2.circle(frame, (x_new, y_new), 8, (0, 0, 255), -1)
    
    # Display the image along with the mask we drew the line on.
    img = cv2.add(frame, mask)
    cv2.imshow('frame', img)
   
    if(cv2.waitKey(10) & 0xFF == 27):
        break
   
    # Now update the previous frame and previous points
    prev_gray = frame_gray.copy()
    prevPts = good_new.reshape(-1, 1, 2)
    
cv2.destroyAllWindows()
cap.release()

Difference between np.zeros(a.shape) and np.zeros_like(a):
Both return a new np with same shape. But zeros_like also maintains same dtype as a. If we don't maintain dtype, sometimes issues may occur

## Dense Optical Flow: Gunner-Farneback Algorithm

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [7]:
cap = cv2.VideoCapture(0)

ret, prev_frame = cap.read()  
prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

hsv_mask = np.zeros_like(prev_frame)
hsv_mask[:, :, 1] = 255 


while True:
    ret, cur_frame = cap.read()
    cur_frame_gray = cv2.cvtColor(cur_frame, cv2.COLOR_BGR2GRAY)
    
    flow = cv2.calcOpticalFlowFarneback(prev_frame_gray, cur_frame_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    
    magnitude, angle = cv2.cartToPolar(flow[:, :, 0], flow[:, :, 1], angleInDegrees = True)
    
    hsv_mask[:, :, 0] = angle/2
    hsv_mask[:, :, 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    
    bgr = cv2.cvtColor(hsv_mask, cv2.COLOR_HSV2BGR)
    
    cv2.imshow('frame', bgr)
    
    if(cv2.waitKey(10) & 0xFF == 27):
        break
    
    prev_frame_gray = cur_frame_gray
        
    
cap.release()
cv2.destroyAllWindows()

### Face detector (Cascade classifier)

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [11]:
cap = cv2.VideoCapture(0)

ret, frame = cap.read()
face_cascade = cv2.CascadeClassifier('./DATA/haarcascades/haarcascade_frontalface_default.xml')
face_rectangles = face_cascade.detectMultiScale(frame)

x, y, w, h = face_rectangles[0]   # Retrieving only first rectangle
track_window = (x, y, w, h)

roi = frame[y:y + h, x:x + w]

hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

roi_histogram = cv2.calcHist([hsv_roi], [0], None, [180], [0, 180])
cv2.normalize(roi_histogram, roi_histogram, 0, 255, cv2.NORM_MINMAX)

termination_criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

while(True):
    
    ret, frame = cap.read()
        
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    dst = cv2.calcBackProject([hsv], [0], roi_histogram, [0, 180], 1)
        
    ret2, track_window = cv2.meanShift(dst, track_window, termination_criteria)
    ret2, track_window = cv2.CamShift(dst, track_window, termination_criteria)
    
        
    x, y, w, h = track_window
        
    img = cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 5)
    cv2.imshow('frame', img)
        
    if(cv2.waitKey() & 0xFF == 27):
        break
    
cap.release()
cv2.destroyAllWindows()